In [1]:
#  word level CTC
# tensorflow version 1.3
# personal note, CTC Tensorflow documentation is not at all clear and intuitive , so if confused refer to any example
# note for decoder, the input has to be in this format 3-D float Tensor sized [max_time x batch_size x num_classes]
# it doesn't support time major option

# from v4 onwards trying to integrate dataset api

# to do :   , graphical loss function , testing on realdataset , seperate training and inference 


# current problems : lousy implementation , for each batch, I am creating a temporary numpy array and padding values ..

# if I use sequence length property of tf.contrib.rnn.static_rnn  , it will do dynamic computation , in other words
# I don't need to calculate differences for padding inputs, just pad to a upper limit, but since its dynamic 
# tf will only unroll till sequence length for each of the inputs 



In [2]:
import tensorflow as tf
from __future__ import print_function
from tensorflow.contrib import rnn
import numpy as np
import scipy.io.wavfile as wav
from python_speech_features import mfcc
from tensorflow.contrib.data import Dataset, Iterator
import time


In [3]:
def audio_to_mfcc(fileurl):
    rate, sig = wav.read(fileurl)
    mfcc_feat = mfcc(sig,rate)
#d_mfcc_feat = delta(mfcc_feat, 2)
#fbank_feat = logfbank(sig,rate)

    return mfcc_feat

In [4]:
inputs = audio_to_mfcc('/home/saurabh/Documents/tf_orange/tf_orange/data/test.wav')

In [5]:
inputs.shape

(299, 13)

In [6]:
def decode_csv(line):
       parsed_line = tf.decode_csv(line, [[""],[""]])
       
    
       
       

       return parsed_line[0] , parsed_line[1]

In [7]:
# we also need a fixed vocabulary 
import re

word_dictionary = {}

with open("/home/saurabh/Documents/tf_orange/tf_orange/words/words.txt") as file:
    for i , line in enumerate(file):
        
        line = line.replace("\n", "")
        word_dictionary[line] = i



In [8]:
print(len(word_dictionary))

8


In [9]:



def word_to_index(sentence):
   
    words = sentence.split(' ')
    index_list=[]
    for word in words:
       
        if word in word_dictionary:
           # print(word)
            index_list.insert(len(index_list) , word_dictionary[word])
            index_list.insert(len(index_list) , word_dictionary[' '])
    index_list.pop()        
    return index_list
  

In [10]:
word_to_index('this is a test')

[0, 6, 1, 6, 2, 6, 3]

In [11]:
def _read_py_function(audio, label):
    audio =audio_to_mfcc(audio)
   
    return audio ,label

In [12]:
dataset = tf.contrib.data.TextLineDataset("/home/saurabh/Documents/tf_orange/tf_orange/data/data.csv")
dataset=dataset.map(decode_csv)



dataset = dataset.map(
    lambda audio, label: tuple(tf.py_func(
        _read_py_function, [audio, label], [tf.double, label.dtype])))




dataset=dataset.batch(2)


In [13]:

iterator = dataset.make_initializable_iterator()

item = iterator.get_next()

In [14]:
# this function is required for CTC Loss
# for it's input , first convert transcrition / ground truth to number representation 

def sparse_tuple_from(sequences, dtype=np.int32):
    """Create a sparse representention of x.
    Args:
        sequences: a list of lists of type dtype where each element is a sequence
    Returns:
        A tuple with (indices, values, shape)
    """
    indices = []
    values = []

    for n, seq in enumerate(sequences):
        indices.extend(zip([n]*len(seq), range(len(seq))))
        values.extend(seq)

    indices = np.asarray(indices, dtype=np.int64)
    values = np.asarray(values, dtype=dtype)
    shape = np.asarray([len(sequences), np.asarray(indices).max(0)[1]+1], dtype=np.int64)

    return indices, values, shape

In [15]:
# Training Parameters
learning_rate = 0.001
training_steps = 20
batch_size = 2
display_step = 200
num_features = 13

# Network Parameters

timesteps = 300 # timesteps
num_hidden = 128 # hidden layer num of features
num_classes = 9 # words.txt , all words plus space word  ( 10001) + CTC symbol (1)




In [16]:
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([batch_size, 2 * num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [17]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell_fw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)
    
    lstm_cell_bw = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    
    # Get lstm cell output
    outputs, _, _ = rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x,
    dtype=tf.float32)
    
    #convert output shape (timesteps * batch * classes ) to (batch*timesteps*classes)
    outputs=tf.transpose( outputs , [1, 0, 2])
    
    
    
  
    res =  tf.matmul(outputs, weights['out']) + biases['out']
    
    return res
   # return tf.nn.softmax(tf.matmul(outputs, weights['out']) + biases['out'])

In [18]:

#prediction = tf.nn.softmax(logits)
inputs = tf.placeholder(tf.float32, [None, None, num_features])
targets = tf.sparse_placeholder(tf.int32)
seq_len = tf.placeholder(tf.int32, [None])




logits = RNN(inputs, weights, biases)

loss =  tf.nn.ctc_loss ( targets, logits , seq_len , time_major = False)
cost = tf.reduce_mean(loss)
optimizer = tf.train.MomentumOptimizer(learning_rate,
                                           0.9).minimize(cost)


# Option 2: tf.contrib.ctc.ctc_beam_search_decoder
    # (it's slower but you'll get better results)
decoder_input = tf.transpose(logits, [1, 0, 2])

decoded, log_prob = tf.nn.ctc_greedy_decoder(decoder_input, seq_len)

    

In [19]:
init = tf.global_variables_initializer()

In [20]:
# need this for decoding word give its index key value 

def keys_of_value(dct, value):
    for k in dct:
        if isinstance(dct[k], list):
            if value in dct[k]:
                return k
        else:
            if value == dct[k]:
                return k

In [21]:
# testing above function 

print(keys_of_value(word_dictionary, 2419))

None


In [22]:
with tf.Session() as sess:
    

    # Run the initializer
    sess.run(init)
    

    


    #print(train_seq_len)
    
    for curr_epoch in range(training_steps):
        
        sess.run(iterator.initializer)
        train_cost = train_ler = 0
        start = time.time()
        
       

        while True: 
            try:
                
                max_len=timesteps
                
                
              
            
           
                elem = sess.run(item)
                print(elem[0].shape)
                print(type(elem[0]))
                
                for label in elem[1]:
                    print (word_to_index(label.decode("utf-8")) )
                
                batch_inputs=elem[0]
                
                batch_lengths=[]
                
            
                for index,test_item in enumerate(batch_inputs):
                
                   # print (test_item.shape)
                   # max_len=max(max_len, test_item.shape[0])
                    batch_lengths.insert(index, test_item.shape[0])
                    
                    
                temp_np_inputs = np.zeros((batch_size , max_len , 13 ))
                    
                for index,test_item in enumerate(batch_inputs):
                
                    if test_item.shape[0] < max_len:
                        print("needs padding")
                        difference = max_len - test_item.shape[0]
                        print("difference : " + str(difference))
                        temp_np_inputs[index] = np.pad(test_item,((0,difference),(0,0)), mode="constant")
        
                print('max_len : ' + str(max_len) )
            
                batch_list = []
                
                print(batch_lengths)
                
                
            #    print(temp_np_inputs[0].shape)
                
            
                for index,j in enumerate(elem[1]):
                    batch_list.insert(index, word_to_index(j.decode("utf-8")))
                    
                
                batch_targets = sparse_tuple_from(batch_list)
                
                
                
             
                
                
            
                feed = {inputs: temp_np_inputs,
                        targets: batch_targets,
                        seq_len: batch_lengths}
	    

                batch_cost, _ = sess.run([cost, optimizer], feed)
                print(batch_cost)
                
               
                
            except tf.errors.OutOfRangeError:
                print("End of training dataset.")
                break
                
    
    # Decoding
    d = sess.run(decoded[0], feed_dict=feed)
   
    dense_decoded = tf.sparse_tensor_to_dense(d, default_value=-1).eval(session=sess)
    
    for i, seq in enumerate(dense_decoded):

        seq = [s for s in seq if s != -1]
       
        str_decoded = ''.join([keys_of_value(word_dictionary, x) for x in seq ])
        print(str_decoded)
    
 
    
    train_cost += batch_cost*batch_size
    print(train_cost)
    
    

(2, 299, 13)
<class 'numpy.ndarray'>
[4, 6, 5, 6, 4, 6, 5]
[5, 6, 5, 6, 5]
needs padding
difference : 1
needs padding
difference : 1
max_len : 300
[299, 299]
2023.16
(2, 299, 13)
<class 'numpy.ndarray'>
[0, 6, 1, 6, 2, 6, 3]
[0, 6, 1, 6, 3]
needs padding
difference : 1
needs padding
difference : 1
max_len : 300
[299, 299]
196.122
End of training dataset.
(2, 299, 13)
<class 'numpy.ndarray'>
[4, 6, 5, 6, 4, 6, 5]
[5, 6, 5, 6, 5]
needs padding
difference : 1
needs padding
difference : 1
max_len : 300
[299, 299]
89.8996
(2, 299, 13)
<class 'numpy.ndarray'>
[0, 6, 1, 6, 2, 6, 3]
[0, 6, 1, 6, 3]
needs padding
difference : 1
needs padding
difference : 1
max_len : 300
[299, 299]
153.368
End of training dataset.
(2, 299, 13)
<class 'numpy.ndarray'>
[4, 6, 5, 6, 4, 6, 5]
[5, 6, 5, 6, 5]
needs padding
difference : 1
needs padding
difference : 1
max_len : 300
[299, 299]
78.1443
(2, 299, 13)
<class 'numpy.ndarray'>
[0, 6, 1, 6, 2, 6, 3]
[0, 6, 1, 6, 3]
needs padding
difference : 1
needs padding
di